### Librerias necesarias

In [1]:
import numpy as np

### Distancia de Minkowski

In [2]:
# Vamos criar uma função pNorm que recebe como parâmetro p e retorna uma função que calcula a pNorma
def pNorm(p):
    """Generates a function to calculate the p-Norm between two points.

    Args:
        p (int): The integer p.

    Returns:
        Dist: A function that calculates the p-Norm.
    """

    def Dist(x,y):
        return np.power(np.power(np.abs(x-y),p).sum(),1/float(p))
    return Dist

In [12]:
# Vamos criar uma RDD com valores numéricos
np.random.seed(42)
numPointsRDD = sc.parallelize(enumerate(np.random.random(size=(10,100))))

In [20]:
# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartPointsRDD = numPointsRDD.cartesian(numPointsRDD)

# Aplique um mapa para transformar nossa RDD em uma RDD de tuplas ((id1,id2), (vetor1,vetor2))
# DICA: primeiro utilize o comando take(1) e imprima o resultado para verificar o formato atual da RDD
cartPointsParesRDD = cartPointsRDD.map(lambda x: ((x[0][0],x[1][0]), (x[0][1],x[1][1])))


# Aplique um mapa para calcular a Distância Euclidiana entre os pares
Euclid = pNorm(2)
distRDD = cartPointsParesRDD.map(lambda x: (x[0], Euclid(x[1][0],x[1][1])))
distRDD

# Encontre a distância máxima, mínima e média, aplicando um mapa que transforma (chave,valor) --> valor
# e utilizando os comandos internos do pyspark para o cálculo da min, max, mean
statRDD = distRDD.map(lambda x: x[1])

minv, maxv, meanv = statRDD.min(), statRDD.max(), statRDD.mean()
print (minv, maxv, meanv)

0.0 4.709048183663605 3.75119168897537


### Similitud de coseno

In [11]:
import numpy as np

In [16]:
from numpy import dot
from numpy.linalg import norm
def DistanciaCoseno(List1,List2):
    print(List1)
    print(List2)
    return dot(List1, List2)/(norm(List1)*norm(List2))

In [20]:
numPointsRDD = sc.parallelize([[[20, 14,10],[8, 11,17]]])

distRDD = numPointsRDD.map(lambda x: (DistanciaCoseno(x[0],x[1])))

print(distRDD.collect())

[0.842658432240182]


[20, 14, 10]
[8, 11, 17]


### Normalización

In [2]:
def nor(v):
    n= v/np.linalg.norm(v)
    return n

np.random.seed(10)
numPointsRDD = sc.parallelize(np.random.randint(10000, size=10000))
numPointsRDD = sc.parallelize([[[0.37454012, 0.95071431,7,77,0.73199394, 0.59865848,22,-222,0.73199394, 0.59865848,22,-222]]])
distRDD = numPointsRDD.map(lambda x: nor(x[0]))
print(distRDD.collect())


[array([ 0.00115302,  0.00292677,  0.0215495 ,  0.23704448,  0.00225344,
        0.00184297,  0.067727  , -0.68342695,  0.00225344,  0.00184297,
        0.067727  , -0.68342695])]


### Estandarización

In [10]:
import statistics as stats
def Estandarizacion(x):
    return [(i-(sum(x)/len(x)))/(stats.variance(x)) for i in x]

In [14]:
dataRDD=sc.parallelize([[7,-11,4],[9,5,-3],[-21,7,9],[-1,30,18]],4)
dataRDD2 = dataRDD.map(lambda x: (str(x),str(Estandarizacion(x))))
print(dataRDD2.collect())
dataRDD2.toDF(["Punto","Punto Estandarizado"]).show()

[('[7, -11, 4]', '[0.07526881720430108, -0.11827956989247312, 0.043010752688172046]'), ('[9, 5, -3]', '[0.1428571428571429, 0.035714285714285726, -0.17857142857142858]'), ('[-21, 7, 9]', '[-0.06872037914691943, 0.030805687203791468, 0.037914691943127965]'), ('[-1, 30, 18]', '[-0.06821282401091405, 0.05866302864938609, 0.00954979536152797]')]
+------------+--------------------+
|       Punto| Punto Estandarizado|
+------------+--------------------+
| [7, -11, 4]|[0.07526881720430...|
|  [9, 5, -3]|[0.14285714285714...|
| [-21, 7, 9]|[-0.0687203791469...|
|[-1, 30, 18]|[-0.0682128240109...|
+------------+--------------------+



### Escalonamiento

In [15]:
def Escalonamiento(x):
    return [(i-min(x))/(max(x)-min(x)) for i in x]

dataRDD=sc.parallelize([[7,-11,4],[9,5,-3],[-21,7,9],[-1,30,18]],4)
dataRDD2 = dataRDD.map(lambda x: (str(x),str(Escalonamiento(x))))
print(dataRDD2.collect())
#Mostrar Dataset
dataRDD2.toDF(["Punto","Punto Escalonado"]).show()

[('[7, -11, 4]', '[1.0, 0.0, 0.8333333333333334]'), ('[9, 5, -3]', '[1.0, 0.6666666666666666, 0.0]'), ('[-21, 7, 9]', '[0.0, 0.9333333333333333, 1.0]'), ('[-1, 30, 18]', '[0.0, 1.0, 0.6129032258064516]')]
+------------+--------------------+
|       Punto|    Punto Escalonado|
+------------+--------------------+
| [7, -11, 4]|[1.0, 0.0, 0.8333...|
|  [9, 5, -3]|[1.0, 0.666666666...|
| [-21, 7, 9]|[0.0, 0.933333333...|
|[-1, 30, 18]|[0.0, 1.0, 0.6129...|
+------------+--------------------+



### Escalada

In [8]:
def Escalada(num, minimo, maximo):
    return ((num - minimo)/(maximo - minimo))

np.random.seed(10)
numPointsRDD = sc.parallelize(np.random.randint(10000, size=10000))
minimo = min(numPointsRDD.collect())
maximo = max(numPointsRDD.collect())
print(minimo, maximo)
numPointsRDD = numPointsRDD.map(lambda x: Escalada(x, minimo, maximo))
numPointsRDD.collect()

1 9999


[0.1288257651530306,
 0.7293458691738348,
 0.13432686537307462,
 0.7291458291658331,
 0.9372874574914983,
 0.48289657931586316,
 0.15193038607721546,
 0.9224844968993798,
 0.9289857971594319,
 0.6400280056011203,
 0.5648129625925185,
 0.4451890378075615,
 0.02380476095219044,
 0.24424884976995398,
 0.21014202840568114,
 0.34156831366273255,
 0.7290458091618324,
 0.05731146229245849,
 0.8036607321464293,
 0.9166833366673335,
 0.6899379875975195,
 0.040808161632326465,
 0.14052810562112422,
 0.0652130426085217,
 0.39317863572714545,
 0.1365273054610922,
 0.8281656331266253,
 0.2699539907981596,
 0.7489497899579916,
 0.1827365473094619,
 0.028205641128225645,
 0.8285657131426285,
 0.7063412682536507,
 0.26535307061412283,
 0.4490898179635927,
 0.7561512302460492,
 0.6159231846369274,
 0.3601720344068814,
 0.06291258251650331,
 0.5191038207641528,
 0.34156831366273255,
 0.16803360672134426,
 0.9646929385877175,
 0.11582316463292658,
 0.6859371874374875,
 0.8988797759551911,
 0.694538907781